In [1]:
import pygad
import numpy as np
from mpmath import fp, hyp2f1
from numpy.linalg import norm
from numpy.random import rand
from numpy import array, inf, vectorize, repeat, power
from numpy import sum as np_sum
import time
from numpy import abs as np_abs

In [2]:
def param_to_spec(param):
    # print(param)
    _A = param[::2]
    _B = param[1::2]
    output = array([(_A+spins)/2, (_A-spins)/2, _B]).transpose().flatten()
    # print(output)
    return output

def g(h, hb, z, zb):
    h12 = 0
    h34 = 0
    hb12 = 0
    hb34 = 0
    output = (1/2 if h == hb else 1)*(z**h*zb**hb*(hyp2f1(h-h12, h+h34, 2*h, z))*(hyp2f1(hb-hb12, hb+hb34, 2*hb, zb)) +
                                    zb**h*z**hb*(hyp2f1(h-h12, h+h34, 2*h, zb))*(hyp2f1(hb-hb12, hb+hb34, 2*hb, z)))
    return fp.mpc(output)


def p(h, hb, c, z, zb):
    output = c*(power(((z-1)*(zb-1)),1/8)*g(h,hb,z,zb) - power(z,1/8)*power(zb,1/8)*g(h,hb,1-z,1-zb))
#     print('p=',output)
    return output
pts = rand(29,2)

vec_p = vectorize(p, excluded=['z', 'zb'])

def e(spec, pts):
   # _A = vec_p(spec[:,0], spec[:,1], spec[:,2], pts[:,0], pts[:,1])
    #_B = ((pts[:,0]-1)*(pts[:,1]-1))**(1/8)-pts[:,0]**(1/8)*pts[:1]**(1/8)
    #output = _A + repeat(_B, len(spec))
    output=array([(np_sum(vec_p(spec[::3], spec[1::3], spec[2::3], z[0], z[1])) +
                  ((z[0]-1)*(z[1]-1))**(1/8)-z[0]**(1/8)*z[1]**(1/8)) for z in pts])
    return output


def e_abs(spec, pts):
    # _A = vec_p(spec[:, 0], spec[:, 1], spec[:, 2], pts[:, 0], pts[:, 1])
    # _B = ((pts[:, 0] - 1) * (pts[:, 1] - 1)) ** (1 / 8) - pts[:, 0] ** (1 / 8) * pts[:1] ** (1 / 8)
    # output = np_abs(np_sum(_A + repeat(_B, len(spec)), axis=1))
    # print(spec)
    output= np_sum(array([(np_abs(vec_p(spec[::3], spec[1::3], spec[2::3], z[0], z[1])) +
                  ((z[0]-1)*(z[1]-1))**(1/8)-z[0]**(1/8)*z[1]**(1/8)) for z in pts]))
#     print('e_abs=',output)
    return output

def A(spec,pts):
    output = norm(e(spec,pts))/np.sum(e_abs(spec,pts))
#     print('A=',output)
    return output

test_spec = array([[2.,2.,2.44e-4],[.5,.5,.25],[2.,0.,.016],[4.,0.,2.2e-4],[6.,0.,1.36e-5]])

spins = array([0,0,2,4,6])

def accuracy_fn(state):
    return A(param_to_spec(state),pts)

def error_fn(state):
    return norm(np.abs(e(param_to_spec(state),pts)))

In [ ]:
def fitness(solution, solution_idx):
    for i in range(len(solution)):
        if solution[i]<0 or solution[i]>6.5:
            fitness=0
            return fitness
    fitness=1/error_fn(solution)
    return fitness

fitness_function = fitness

num_generations = 100
num_parents_mating = 30

sol_per_pop = 100
num_genes = 10

init_range_low = 0
init_range_high = 6.5

parent_selection_type = "rank"
keep_parents = 10

crossover_type = "uniform"
crossover_probability=0.8
mutation_type = "random"
mutation_probability = 0.8

initial_pop=[]
for i in range(sol_per_pop):
        initial_pop.append([4., rand(), 1., rand(), 2., rand(), 4., rand(), 6., rand()])

def call(x):
    index=x.generations_completed
    if index%10==0:
            solution, solution_fitness, solution_idx = x.best_solution(ga_instance.last_generation_fitness)
            print('Generation:{}, best fitness:{}'.format(x.generations_completed, solution_fitness))
    
ga_instance = pygad.GA(num_generations=num_generations,
                       num_parents_mating=num_parents_mating,
                       fitness_func=fitness_function,
                       sol_per_pop=sol_per_pop,
                       num_genes=num_genes,
                       initial_population=initial_pop,
                       #init_range_low=init_range_low,
                       #init_range_high=init_range_high,
                       parent_selection_type=parent_selection_type,
                       keep_parents=keep_parents,
                       crossover_type=crossover_type,
                       mutation_type=mutation_type,
                       mutation_probability=mutation_probability,
                       random_mutation_min_val=-0.1,
                       random_mutation_max_val=0.1,
                       on_generation=call)
ga_instance.run()
ga_instance.plot_fitness()

# Returning the details of the best solution.
solution, solution_fitness, solution_idx = ga_instance.best_solution(ga_instance.last_generation_fitness)
print("Parameters of the best solution : {solution}".format(solution=solution))
print("Fitness value of the best solution = {solution_fitness}".format(solution_fitness=solution_fitness))
print("Best solution error:", error_fn(solution))
print("Best solution accuracy:", accuracy_fn(solution))

Generation:10, best fitness:0.039301707506510485
